In [6]:
#!pip install pytesseract
#!pip install easyocr

In [31]:
import re
#import constants
import os
import pandas as pd
import multiprocessing
import time
from time import time as timer
#from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
#import requests
import urllib
from PIL import Image
import easyocr
import cv2

In [14]:
def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except Exception as e:
            print(e)
            time.sleep(delay)
    
    create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images


In [12]:
def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        return


In [20]:
reader = easyocr.Reader(['en'])

In [32]:
# # loading the image
# link_1 = "https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg"
# download_image(link_1, "./images")


In [33]:
# result_1 = reader.readtext("./images/61I9XdN6OFL.jpg", detail = 0)
# result_1

In [26]:
# loading the image
link_2 = "https://m.media-amazon.com/images/I/71gSRbyXmoL.jpg"
download_image(link_2, "./images")

In [57]:
# STEP 1 : Normalization
# Normalization of the image without adaptive threshold

# Read color image
image = cv2.imread("./images/71gSRbyXmoL.jpg")

# Convert image to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Normalising grayscale image
normalized_gray_image = cv2.normalize(
    gray_image, 
    None, 
    alpha=0, 
    beta=255, 
    norm_type=cv2.NORM_MINMAX)

cv2.imwrite('./images/original_image.png', image)
cv2.imwrite('./images/normalized_image.jpg', normalized_gray_image)


True

In [66]:
# STEP 2 : Skew correction function
# HAVEN'T INCORPORATED THIS FOR NOW!
def deskew(image):
    co_ords = np.column_stack(np.where(image > 0))

    # Check if co_ords is empty
    if co_ords.size == 0:
        print("No non-zero pixels found in the image for skew correction.")
        return image

    angle = cv2.minAreaRect(co_ords)[-1]
    print(angle)

    if angle < -45:
        angle = -(90 + angle)
    else:
        pass
        #angle = -angle

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    return rotated

# # Convert to grayscale before deskewing (if working with color images)
# gray_image_for_skew = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply skew correction
# skew_corrected_img = deskew(gray_image_for_skew)

# # Save the skew corrected image
# cv2.imwrite('./images/skew_corrected_img.jpg', skew_corrected_img)


In [73]:
def set_image_dpi(file_path, save_directory):
    # Open the image
    im = Image.open(file_path)

    # Resize the image
    length_x, width_y = im.size
    factor = min(1, float(1024.0 / length_x))
    size = int(factor * length_x), int(factor * width_y)
    im_resized = im.resize(size, Image.Resampling.LANCZOS)

    # Ensure the save directory exists
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    # Construct the full path for the new file
    file_name = os.path.basename(file_path)  # Get the original file name
    temp_filename = os.path.join(save_directory, f"resized_{file_name}")

    # Save the resized image with the specified DPI
    im_resized.save(temp_filename, dpi=(300, 300))

    return temp_filename

# Example usage
save_directory = './images'  # Replace with your desired directory
temp_filename = set_image_dpi('./images/normalized_image.jpg', save_directory)
print(f"Image saved at: {temp_filename}")


Image saved at: ./images/resized_normalized_image.jpg


In [56]:
# Using automatic threshold decider to convert an already grayscale image to increase contrast

def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

grayscale_threshold = thresholding(normalized_gray_image)
cv2.imwrite('./images/grayscale_threshold.jpg', grayscale_threshold)

True

In [54]:
result_2 = reader.readtext("./images/normalized_image.jpg", detail = 0)
result_2

['E leLeeeE=_',
 'Rcia',
 'TEARRIFICY [',
 'LEBENSMITTELECHT',
 'GEPRAGTES',
 'DESIGN',
 'AS YoULIKE',
 'BEST',
 'Designed in',
 'V',
 'VV',
 'cakea',
 'Y',
 'Bextin]',
 'LIZENZIERTE UND GESCHITZTE DESIGNS',
 '1Cup',
 'DAY',
 'WITH',
 'cecieics']

In [64]:
result_3 = reader.readtext("./images/skew_corrected_img.jpg", detail = 0)
result_3

['1',
 'p',
 'Je',
 '1',
 '9',
 '13',
 '[',
 '{',
 '8',
 ')',
 '0',
 '2',
 'L',
 '1',
 '2',
 '3']